In [ ]:
import os
import glob

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
import pandas as pd
import numpy as np
from PIL import Image

import umap

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA

## Clean data

In [ ]:
# decals = pd.read_parquet('../zoobot_private/gz_decals_auto_posteriors.parquet', columns=['iauname'])
# # # decals = pd.read_parquet('../zoobot_private/gz_decals_auto_posteriors.parquet')
# safe_galaxies = set(decals['iauname'])

In [ ]:
file_locs = glob.glob('/media/walml/beta1/cnn_features/*_output_*.csv')
len(file_locs)

0

In [ ]:
image_format = 'png'
    
for loc in file_locs:
    print(loc)
    clean_loc = loc.replace('_output_', '_cleaned_').replace('.csv', '.parquet')
    if not os.path.isfile(clean_loc):

        features_df = pd.read_csv(loc)
        feature_cols = [col for col in features_df if col.startswith('feat')]

        for col in feature_cols:
            features_df[col] = features_df[col].apply(lambda x: float(x.replace('[', '').replace(']', '')))  # extract from list e.g. [0.1456] to 0.1456

        features_df['png_loc'] = features_df['image_loc'].str.replace('/raid/scratch/walml/galaxy_zoo/decals/png', '/media/walml/beta1/decals/png_native/dr5')

    #     assert all([os.path.isfile(x) for x in features_df['filename']])
        del features_df['image_loc']
        features_df['iauname'] = list(features_df['png_loc'].apply(lambda x: x.split('/')[-1].replace(f'.{image_format}', '')))

    #         features_df = features_df[features_df['objid'].isin(safe_galaxies)]

        # must be indexed by the names of the png files for astronomaly to know which image has which features
        # cast to list to avoid pandas renaming index to image_loc - too clever for its own good

        features_df.to_parquet(clean_loc)

In [ ]:
cleaned_locs = glob.glob('/media/walml/beta1/cnn_features/*_cleaned_*.parquet')

data = []
for loc in cleaned_locs:
    data.append(pd.read_parquet(loc))
features_df = pd.concat(data)
print(len(features_df))

ValueError: No objects to concatenate

In [ ]:
features_df.sort_values('png_loc').to_parquet('/media/walml/beta1/cnn_features/cnn_features_concat.parquet', index=False)
# will merge this with the carefully-cut list of <=dr5 galaxies below to apply those cuts
# see select_safe_catalog.ipynb